In [1]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
from enum import Enum
RI = np.array([0,0,0.58,0.90,1.12,1.24,1.32,1.41,1.45,1.49,1.51,1.48,1.56,1.57,1.58])

#subcriteria
df =pd.read_csv("WesternGhatData/1Elevation.csv")
elevation= df.to_numpy()

df =pd.read_csv("WesternGhatData/2Slope.csv")
slope= df.to_numpy()

df =pd.read_csv("WesternGhatData/3DistancefromRiver.csv")
river= df.to_numpy()

df =pd.read_csv("WesternGhatData/4Drainage.csv")
drainage= df.to_numpy()

df =pd.read_csv("WesternGhatData/5FlowAccumulation.csv")
flow= df.to_numpy()

df =pd.read_csv("WesternGhatData/6TWI.csv")
twi= df.to_numpy()

df =pd.read_csv("WesternGhatData/7Rainfall.csv")
rainfall= df.to_numpy()

df =pd.read_csv("WesternGhatData/8Landuse.csv")
landuse= df.to_numpy()

df =pd.read_csv("WesternGhatData/9SoilTexture.csv")
soil= df.to_numpy()

df =pd.read_csv("WesternGhatData/10TRI.csv")
tri= df.to_numpy()

df =pd.read_csv("WesternGhatData/11Geology.csv")
geology= df.to_numpy()

# pairwise comparison matrix of main criterias
# data = np.array([[1   ,2   ,2   ,3   ,4   ,5   ,5   ,6   ,7   ,8   ,8],
#                    [1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,6   ,7   ,8   ,8],
#                    [1/2 ,1/2 ,1   ,2   ,3   ,4   ,4   ,5   ,6   ,7   ,7],
#                    [1/3 ,1/3 ,1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,7   ,8],
#                    [1/4 ,1/3 ,1/3 ,1/2 ,1   ,2   ,2   ,3   ,4   ,5   ,6],
#                    [1/5 ,1/4 ,1/4 ,1/3 ,1/2 ,1   ,2   ,4   ,5   ,7   ,7],
#                    [1/5 ,1/5 ,1/4 ,1/3 ,1/2 ,1/2 ,1   ,2   ,3   ,5   ,6],
#                    [1/6 ,1/6 ,1/5 ,1/4 ,1/3 ,1/4 ,1/2 ,1   ,2   ,4   ,5],
#                    [1/7 ,1/7 ,1/6 ,1/5 ,1/4 ,1/5 ,1/3 ,1/2 ,1   ,3   ,4],
#                    [1/8 ,1/8 ,1/7 ,1/7 ,1/5 ,1/7 ,1/5 ,1/4 ,1/3 ,1   ,3],
#                    [1/8 ,1/8 ,1/7 ,1/8 ,1/6 ,1/7 ,1/6 ,1/5 ,1/4 ,1/3 ,1]]
#                   )

data = np.array([
                [1   ,2   ,2   ,3   ,4   ,5   ,5   ,6   ,7   ,8 ],
                [1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,6   ,7   ,8 ],
                [1/2 ,1/2 ,1   ,2   ,3   ,4   ,4   ,5   ,6   ,7 ],
                [1/3 ,1/3 ,1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,7 ],
                [1/4 ,1/3 ,1/3 ,1/2 ,1   ,2   ,2   ,3   ,4   ,5 ],
                [1/5 ,1/4 ,1/4 ,1/3 ,1/2 ,1   ,2   ,4   ,5   ,7 ],
                [1/5 ,1/5 ,1/4 ,1/3 ,1/2 ,1/2 ,1   ,2   ,3   ,5 ],
                [1/6 ,1/6 ,1/5 ,1/4 ,1/3 ,1/4 ,1/2 ,1   ,2   ,4 ],
                [1/7 ,1/7 ,1/6 ,1/5 ,1/4 ,1/5 ,1/3 ,1/2 ,1   ,3 ],
                [1/8 ,1/8 ,1/7 ,1/7 ,1/5 ,1/7 ,1/5 ,1/4 ,1/3 ,1 ]
                
                 ])

class memfunc(Enum):
    TRIANGULAR = 3
    TRAPEZOIDAL = 4

FileNotFoundError: [Errno 2] No such file or directory: 'WesternGhatData/1Elevation.csv'

In [ ]:
# change this to change the membership func
size3d=memfunc.TRIANGULAR.value

#converts normal into fuzzyarray
def fuzzify(data):
    n = data[0].size
    mainarr = np.full((n,n,size3d),0,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            if (data[i][j]>=1):
                mainarr[i][j][0]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]-1
                mainarr[i][j][1]= data[i][j]
                mainarr[i][j][2]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]+1
                if(size3d == memfunc.TRAPEZOIDAL.value):
                    mainarr[i][j][3]= data[i][j]+1 if(data[i][j]==8) else data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]+2
            else:
                mainarr[i][j]= np.reciprocal(np.flip(mainarr[j][i]))
    return mainarr

In [ ]:
# AHP CRITERIA WEIGHTS
def criteriaWeights(mainarr):
    n = mainarr[0].size
    colsum = mainarr.sum(axis=0)
    normalizedform = np.array(mainarr)
    for i in range(0,n):
        normalizedform[i] = np.divide(mainarr[i] , colsum)
    criteriaweights = normalizedform.sum(axis=1)/n
    return criteriaweights
# FUZZY AHP CRITERIA WEIGHTS
def FuzzycriteriaWeights(mainarr):
    n = mainarr[0].size if (np.ndim(mainarr)==2) else int(mainarr[0].size/size3d)
    fuzzyGM = np.full((n,size3d),1,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            fuzzyGM[i] = np.multiply(fuzzyGM[i],mainarr[i][j])
        fuzzyGM[i]=np.power(fuzzyGM[i],1/n)
    fuzzyWeights=np.full((n,size3d),1,dtype=float)
    for i in range(0,n):
        fuzzyWeights[i]=np.multiply(fuzzyGM[i],np.reciprocal(np.flip(fuzzyGM.sum(axis=0),0)))
    weights = np.average(fuzzyWeights,axis=1)
    weights = np.divide(weights,weights.sum(axis=0))
    return weights

In [ ]:
def consistency(mainarr):
    n = int(mainarr[0].size)
    fuzzyArr = fuzzify(mainarr)
    c1 = np.array(mainarr)
    fc1 = np.array(mainarr)
    for i in range(0,n):
        c1[i] = np.multiply(mainarr[i],criteriaWeights((mainarr)))
        fc1[i] = np.multiply(mainarr[i],FuzzycriteriaWeights((mainarr)))
    lamda = np.divide(c1.sum(axis=1),criteriaWeights((mainarr))).sum(axis=0)/n
    flamda = (np.divide((fc1.sum(axis=1)),FuzzycriteriaWeights((mainarr)))).sum(axis=0)/n
    ci = (lamda-n)/(n-1)
    fci = (flamda-n)/(n-1)
    CR = ci / RI[n-1]
    FCR = fci / RI[n-1]

    if(CR<0.1 and FCR<0.1):
        print("CR: ",CR," VALUES ARE CONSISTENT - go ahead")
        print("cwAHP",criteriaWeights(mainarr))
        print("FUZZY CR: ",FCR," VALUES ARE CONSISTENT - go ahead")
        print("cwFuzzyAHP",FuzzycriteriaWeights(fuzzyArr))
    else:
        print("CR: ",CR,"criteria weights are not consistent reEnter the values")
        print("FUZZY CR: ",FCR," VALUES ARE CONSISTENT - go ahead")
    
    return FuzzycriteriaWeights(fuzzyArr)

In [ ]:
consistency(data)

CR:  0.06480286359154262  VALUES ARE CONSISTENT - go ahead
cwAHP [0.23568782 0.19472159 0.15248456 0.11228755 0.07805507 0.07511335
 0.05317075 0.0381161  0.02810247 0.01852672 0.01373403]
FUZZY CR:  0.061794976010377985  VALUES ARE CONSISTENT - go ahead
cwFuzzyAHP [0.23105305 0.19836926 0.15984415 0.11672598 0.08166557 0.07001366
 0.05301382 0.03564976 0.02576789 0.01598071 0.01191615]


array([0.23105305, 0.19836926, 0.15984415, 0.11672598, 0.08166557,
       0.07001366, 0.05301382, 0.03564976, 0.02576789, 0.01598071,
       0.01191615])

In [ ]:
print(np.multiply(consistency(elevation),FuzzycriteriaWeights(fuzzify(data))[0]),"Global elevation\n\n")
print(np.multiply(consistency(slope),FuzzycriteriaWeights(fuzzify(data))[1]),"Global slope\n\n")
print(np.multiply(consistency(river),FuzzycriteriaWeights(fuzzify(data))[2]),"Global river\n\n")
print(np.multiply(consistency(drainage),FuzzycriteriaWeights(fuzzify(data))[3]),"Global drainage\n\n")
print(np.multiply(consistency(flow),FuzzycriteriaWeights(fuzzify(data))[4]),"Global flow\n\n")
print(np.multiply(consistency(twi),FuzzycriteriaWeights(fuzzify(data))[5]),"Global twi\n\n")
print(np.multiply(consistency(rainfall),FuzzycriteriaWeights(fuzzify(data))[6]),"Global rainfall\n\n")
print(np.multiply(consistency(landuse),FuzzycriteriaWeights(fuzzify(data))[7]),"Global landuse\n\n")
print(np.multiply(consistency(soil),FuzzycriteriaWeights(fuzzify(data))[8]),"Global soil\n\n")
print(np.multiply(consistency(tri),FuzzycriteriaWeights(fuzzify(data))[9]),"Global tri\n\n")
print(np.multiply(consistency(geology),FuzzycriteriaWeights(fuzzify(data))[10]),"Global geology\n\n")


CR:  0.02159551613867609  VALUES ARE CONSISTENT - go ahead
cwAHP [0.51810572 0.21808442 0.13925169 0.08397522 0.04058294]
FUZZY CR:  0.021324860097975327  VALUES ARE CONSISTENT - go ahead
cwFuzzyAHP [0.50931485 0.2210592  0.14425756 0.0856766  0.03969178]
[0.11767875 0.0510764  0.03333115 0.01979584 0.00917091] Global elevation


CR:  0.029474590206579408  VALUES ARE CONSISTENT - go ahead
cwAHP [0.49854271 0.27594514 0.11266966 0.0698327  0.0430098 ]
FUZZY CR:  0.02932557462709004  VALUES ARE CONSISTENT - go ahead
cwFuzzyAHP [0.49509351 0.27578642 0.114      0.07099814 0.04412193]
[0.09821133 0.05470755 0.0226141  0.01408385 0.00875244] Global slope


CR:  0.04215341953280287  VALUES ARE CONSISTENT - go ahead
cwAHP [0.39978196 0.25794431 0.19703637 0.10227381 0.04296354]
FUZZY CR:  0.04156288231905169  VALUES ARE CONSISTENT - go ahead
cwFuzzyAHP [0.3919398  0.2682075  0.19668145 0.1013873  0.04178395]
[0.06264928 0.0428714  0.03143838 0.01620617 0.00667892] Global river


CR:  0.013463